In [1]:
from autotst.reaction import *

In [2]:
from autotst.conformer.ga import *

In [7]:
from ase.calculators.emt import EMT
rxn.ts.ase_ts.set_calculator(EMT())
perform_ga(rxn, max_generations=1)

ga.py:77 perform_ga INFO No initial population provided, creating one using base parameters...
utilities.py:82 create_initial_population INFO Creating initial population of 100 individuals from random guesses
utilities.py:95 create_initial_population INFO The object given is a `AutoTST_Reaction` object


      Step     Time          Energy         fmax
BFGS:    0 15:40:01        5.628461        4.6813
BFGS:    1 15:40:01        4.978406        3.6036
BFGS:    2 15:40:02        4.653835        2.6227
BFGS:    3 15:40:02        4.456317        1.6370
BFGS:    4 15:40:02        4.341158        1.0391
BFGS:    5 15:40:02        4.284916        0.6924
BFGS:    6 15:40:02        4.242861        0.8033
BFGS:    7 15:40:03        4.196537        0.6593
BFGS:    8 15:40:03        4.164264        0.4494
BFGS:    9 15:40:03        4.153814        0.2138
BFGS:   10 15:40:03        4.152142        0.1586
BFGS:   11 15:40:03        4.150293        0.1214
BFGS:   12 15:40:04        4.148450        0.1708
BFGS:   13 15:40:04        4.144570        0.2393
BFGS:   14 15:40:04        4.137118        0.4187
BFGS:   15 15:40:04        4.129024        0.5515
BFGS:   16 15:40:04        4.120553        0.6620
BFGS:   17 15:40:05        4.112890        0.7624
BFGS:   18 15:40:05        4.097917        0.5194
B

KeyboardInterrupt: 

In [3]:
mol = AutoTST_Molecule("COCC(CC)CON")

mol

<AutoTST Molecule "COCC(CC)CON">

In [4]:
from ase.calculators.gaussian import Gaussian


In [5]:
mol.rmg_molecule.updateMultiplicity()
mol.rmg_molecule.multiplicity

1

In [6]:
calcs = {}
for torsion in mol.torsions:
    mem="5GB"
    nprocshared=20
    scratch="."
    method="m062x"
    basis="6-311+g(2df,2p)"
    string = ""
    for bond in mol.bonds:
        i,j =  bond.indices

        string += "B {} {} F\n".format(i+1,j+1)

    for angle in mol.angles:
        i,j,k =  angle.indices

        string += "A {} {} {} F\n".format(i+1,j+1,k+1)
    i,j,k,l = torsion.indices
    string += "D {} {} {} {} S 36 10.0".format(i+1,j+1,k+1,l+1)
    
    smiles = mol.rmg_molecule.toSMILES()
    
    label = Chem.rdinchi.InchiToInchiKey(Chem.MolToInchi(Chem.MolFromSmiles(smiles))).strip("-N")
    label += "_tor{}{}".format(j,k)
    calc = Gaussian(mem=mem,
                    nprocshared=nprocshared,
                    label=label,
                    scratch=scratch,
                    method=method,
                    basis=basis,
                    extra="Opt=(ModRedun)",
                    multiplicity = mol.rmg_molecule.multiplicity,
                    addsec=[string])
    
    del calc.parameters['force']
    calcs[(i,j)] = calc
    print calc.addsec[0][-20:]
    print j,k
calcs

D 3 1 6 20 S 36 10.0
0 5
D 6 1 3 10 S 36 10.0
0 2
D 7 2 4 14 S 36 10.0
1 3
D 4 2 7 19 S 36 10.0
1 6
D 17 5 8 9 S 36 10.0
4 7
 20 6 9 24 S 36 10.0
5 8
 19 7 9 24 S 36 10.0
6 8
 22 8 9 24 S 36 10.0
7 8


{(2, 0): <ase.calculators.gaussian.Gaussian at 0x1816ef1c10>,
 (3, 1): <ase.calculators.gaussian.Gaussian at 0x1816ef1d10>,
 (5, 0): <ase.calculators.gaussian.Gaussian at 0x180f403910>,
 (6, 1): <ase.calculators.gaussian.Gaussian at 0x1816ef1c50>,
 (16, 4): <ase.calculators.gaussian.Gaussian at 0x180f403bd0>,
 (18, 6): <ase.calculators.gaussian.Gaussian at 0x180f3ebb50>,
 (19, 5): <ase.calculators.gaussian.Gaussian at 0x10cc6ead0>,
 (21, 7): <ase.calculators.gaussian.Gaussian at 0x180f3ebbd0>}

In [146]:
string = ""
for bond in mol.bonds:
    i,j =  bond.indices
    
    string += "B {} {}\n".format(i+1,j+1)
    
for angle in mol.angles:
    i,j,k =  angle.indices
    
    string += "A {} {} {}\n".format(i+1,j+1,k+1)
    
torsion = mol.torsions[3]
for tor in mol.torsions:
    i,j,k,l =  tor.indices
    
    string += "D {} {} {} {}\n".format(i+1,j+1,k+1,l+1)
    
tor = mol.torsions[0]

i,j,k,l = tor.indices

string += "D {} {} {} {} S 36 10.0".format(i+1,j+1,k+1,l+1)
print string
    


B 1 6
B 1 3
B 2 4
B 2 7
B 3 11
B 3 10
B 4 13
B 4 12
B 4 14
B 5 15
B 5 16
B 5 17
B 5 8
B 6 21
B 6 9
B 6 20
B 7 18
B 7 9
B 7 19
B 8 23
B 8 22
B 8 9
B 9 24
A 1 6 21
A 1 6 9
A 1 6 20
A 1 3 11
A 1 3 10
A 2 4 13
A 2 4 12
A 2 4 14
A 2 7 18
A 2 7 9
A 2 7 19
A 3 1 6
A 4 2 7
A 5 8 23
A 5 8 22
A 5 8 9
A 6 9 7
A 6 9 8
A 6 9 24
A 7 9 8
A 7 9 24
A 8 5 15
A 8 5 16
A 8 5 17
A 8 9 24
A 9 6 21
A 9 6 20
A 9 7 18
A 9 7 19
A 9 8 23
A 9 8 22
A 10 3 11
A 12 4 13
A 12 4 14
A 13 4 14
A 15 5 16
A 15 5 17
A 16 5 17
A 18 7 19
A 20 6 21
A 22 8 23
D 3 1 6 20
D 6 1 3 10
D 7 2 4 14
D 4 2 7 19
D 17 5 8 9
D 20 6 9 24
D 19 7 9 24
D 22 8 9 24
D 3 1 6 20 S 36 10.0


In [141]:
calc.addsec = [string]

In [142]:
mol.ase_molecule.set_calculator(calc)
calc

In [143]:
calc.write_input(mol.ase_molecule)

In [8]:
from cclib.io import ccread

In [121]:
p = ccread("/Users/nathan/Code/RMG-Py/examples/cantherm/species/C2H5/ethyl_scan_1.log")

In [122]:
from ase.io.gaussian import read_gaussian_out

In [123]:
p.grads

array([[[-1.654000e-05,  3.631950e-04, -7.500600e-05],
        [ 2.975800e-05, -4.161160e-04,  2.785890e-04],
        [ 1.279587e-03, -2.609100e-05, -8.358020e-04],
        ...,
        [-1.010999e-03,  5.543350e-04,  4.837710e-04],
        [ 6.038400e-05, -9.632850e-04,  4.999450e-04],
        [ 9.286480e-04,  6.866970e-04,  4.840720e-04]],

       [[-8.950000e-07,  1.243900e-05, -2.169000e-06],
        [ 2.972000e-06, -2.131000e-06,  5.323000e-06],
        [ 7.868000e-06,  6.440000e-07, -5.431000e-06],
        ...,
        [-7.205000e-06, -5.641000e-06,  4.956000e-06],
        [ 3.240000e-07, -8.610000e-06, -2.070000e-06],
        [ 5.616000e-06, -1.291000e-06,  4.927000e-06]],

       [[ 1.304431e-03,  5.618860e-04,  5.313870e-04],
        [-5.050658e-03, -8.582106e-03, -3.372790e-04],
        [ 2.050050e-04,  9.006720e-04, -5.452400e-05],
        ...,
        [ 2.628358e-03,  6.229449e-03, -1.449180e-04],
        [ 3.606635e-03, -1.609866e-03,  5.874590e-04],
        [-2.479451e-03

In [124]:

for i in p.scfvalues:
    print len(i)

AttributeError: 'ccData_optdone_bool' object has no attribute 'scfvalues'

In [125]:
len(p.scfenergies)

132

In [126]:
len(p.scfvalues)

AttributeError: 'ccData_optdone_bool' object has no attribute 'scfvalues'

In [127]:
len(p.scftargets) 

132

In [128]:
smallest = max(p.scfenergies) +1
print smallest
print
results = []
print smallest
for i in p.scfenergies:
    #print smallest
    if i < smallest:
        smallest = i
    else:
        results.append(smallest)
        print smallest
        print
        smallest= max(p.scfenergies) + 1
# adding the last one which should be a converged geometry 
results.append(smallest)
        
        
print results

-2153.3044013671765

-2153.3044013671765
-2154.3281820822986

-2154.327539490883

-2154.3258523904524

-2154.325738864554

-2154.3274140001386

-2154.3281804332887

-2154.3276886800227

-2154.3264677405505

-2154.325565038226

-2154.3256003286715

-2154.3265430480583

-2154.3277139457937

-2154.328181470042

-2154.3275171421724

-2154.325916144006

-2154.3256322585107

-2154.327204189475

-2154.328155956648

-2154.3278235260414

-2154.326637289248

-2154.3256144921975

-2154.325568785234

-2154.326514141404

-2154.327740789825

-2154.328173875345

-2154.327341479076

-2154.3257200778135

-2154.3258008466464

-2154.3273979508635

-2154.3281728086586

-2154.3277963228197

-2154.3266576297583

-2154.325649268347

-2154.3255292579756

-2154.3263546772455

-2154.3275978375345

-2154.328182052366

[-2154.3281820822986, -2154.327539490883, -2154.3258523904524, -2154.325738864554, -2154.3274140001386, -2154.3281804332887, -2154.3276886800227, -2154.3264677405505, -2154.325565038226, -2154.3256

In [129]:
print len(p.converged_geometries)
for geo in p.converged_geometries:
    print geo


38
[[ 7.952340e-01  0.000000e+00 -1.866500e-02]
 [-6.949120e-01  7.400000e-05 -1.623000e-03]
 [ 1.354765e+00  9.279580e-01  4.140300e-02]
 [ 1.354410e+00 -9.281950e-01  4.136900e-02]
 [-1.108107e+00 -8.852590e-01 -4.987100e-01]
 [-1.095049e+00 -5.241000e-03  1.027529e+00]
 [-1.107946e+00  8.902970e-01 -4.898630e-01]]
[[-7.952620e-01  0.000000e+00 -1.826100e-02]
 [ 6.949360e-01  2.474000e-03 -1.350000e-03]
 [-1.348822e+00 -9.319980e-01  3.978800e-02]
 [-1.360729e+00  9.240990e-01  4.038500e-02]
 [ 1.105635e+00  9.614360e-01 -3.341470e-01]
 [ 1.094859e+00 -1.806860e-01  1.011161e+00]
 [ 1.111014e+00 -7.876930e-01 -6.395200e-01]]
[[-7.952900e-01 -3.300000e-05 -9.515000e-03]
 [ 6.951330e-01  4.614000e-03 -5.410000e-04]
 [-1.345081e+00 -9.356420e-01  2.052600e-02]
 [-1.367104e+00  9.210030e-01  2.092500e-02]
 [ 1.102092e+00  1.011961e+00 -1.303940e-01]
 [ 1.100771e+00 -3.926270e-01  9.445540e-01]
 [ 1.110262e+00 -6.321830e-01 -7.952720e-01]]
[[ 7.952830e-01 -3.400000e-05 -8.317000e-03]
 [-6

In [130]:
p.converged_geometries[0] - p.converged_geometries[-1]

array([[-8.0e-06, -4.0e-06,  1.0e-05],
       [ 1.0e-05,  0.0e+00, -7.0e-06],
       [-1.2e-05, -2.1e-05, -2.0e-05],
       [-2.3e-05,  7.0e-06, -4.1e-05],
       [-4.8e-05,  1.5e-05,  4.2e-05],
       [ 4.6e-05,  2.4e-05, -4.0e-06],
       [ 3.0e-05,  3.0e-06,  3.0e-06]])

In [131]:
for i in range(len(p.converged_geometries)):
    print ((p.converged_geometries[0] - p.converged_geometries[i]) ** 2).mean()

0.0
2.2361785951156183
2.2372200517484764
0.8631508022109523
0.8192520468690001
0.7752643505269999
1.9041638648335235
1.9053524034893812
1.9062650730910473
0.3218934734875238
0.36447539348961905
0.40670249088342847
2.231269411482381
2.230848054531524
2.2314282732979045
0.3553446730493333
0.3349787934678095
0.3282672060089523
1.9082380140516189
1.9089210061472381
1.909269982201524
0.5429222768034762
0.5145154097489046
0.48054217357171425
2.231364012277762
2.232311123100333
2.234132106030571
0.6640481306259048
0.7221593862377144
0.7694770570174285
1.9030593875965713
1.9032922272656185
1.9035831747186667
0.03163014768933333
0.014825933394285713
0.004005242120142857
5.472380952384401e-10
5.472380952384401e-10


In [132]:
p.converged_geometries[-1]

array([[ 7.952420e-01,  4.000000e-06, -1.867500e-02],
       [-6.949220e-01,  7.400000e-05, -1.616000e-03],
       [ 1.354777e+00,  9.279790e-01,  4.142300e-02],
       [ 1.354433e+00, -9.282020e-01,  4.141000e-02],
       [-1.108059e+00, -8.852740e-01, -4.987520e-01],
       [-1.095095e+00, -5.265000e-03,  1.027533e+00],
       [-1.107976e+00,  8.902940e-01, -4.898660e-01]])

In [133]:
c = calcs[(2,0)]
c.label

'RUZNHLODSVCOOY-UHFFFAOYSA_tor05'

In [134]:
mol.torsions[0].indices

(2, 0, 5, 19)

In [135]:
for i in range(10):
    if i == 5 : pass
    print i

0
1
2
3
4
5
6
7
8
9


In [136]:
results[0] - results[-1]

-1.0237807151220295

In [137]:
l = c.label + ".log"
l.replace(".log", "-failed.log")

'RUZNHLODSVCOOY-UHFFFAOYSA_tor05-failed.log'

In [138]:
for r in results: print r

-2154.3281820822986
-2154.327539490883
-2154.3258523904524
-2154.325738864554
-2154.3274140001386
-2154.3281804332887
-2154.3276886800227
-2154.3264677405505
-2154.325565038226
-2154.3256003286715
-2154.3265430480583
-2154.3277139457937
-2154.328181470042
-2154.3275171421724
-2154.325916144006
-2154.3256322585107
-2154.327204189475
-2154.328155956648
-2154.3278235260414
-2154.326637289248
-2154.3256144921975
-2154.325568785234
-2154.326514141404
-2154.327740789825
-2154.328173875345
-2154.327341479076
-2154.3257200778135
-2154.3258008466464
-2154.3273979508635
-2154.3281728086586
-2154.3277963228197
-2154.3266576297583
-2154.325649268347
-2154.3255292579756
-2154.3263546772455
-2154.3275978375345
-2154.328182052366
-2153.3044013671765


In [139]:
for e in p.scfenergies[p.optstatus == 4]:
    print e

-2154.3281820822986
-2154.327539490883
-2154.3258523904524
-2154.325738864554
-2154.3274140001386
-2154.3281804332887
-2154.3276886800227
-2154.3264677405505
-2154.325565038226
-2154.3256003286715
-2154.3265430480583
-2154.3277139457937
-2154.328181470042
-2154.3275171421724
-2154.325916144006
-2154.3256322585107
-2154.327204189475
-2154.328155956648
-2154.3278235260414
-2154.326637289248
-2154.3256144921975
-2154.325568785234
-2154.326514141404
-2154.327740789825
-2154.328173875345
-2154.327341479076
-2154.3257200778135
-2154.3258008466464
-2154.3273979508635
-2154.3281728086586
-2154.3277963228197
-2154.3266576297583
-2154.325649268347
-2154.3255292579756
-2154.3263546772455
-2154.3275978375345
-2154.328182052366


In [92]:
len(p.scfenergies)

144

In [117]:
p.optstatus

array([1, 4, 1, 0, 4, 1, 0, 4, 1, 0, 0, 4, 1, 0, 0, 4, 1, 0, 0, 4, 1, 0,
       4, 1, 0, 4, 1, 0, 4, 1, 4, 1, 4, 1, 4, 1, 4, 1, 4, 1, 4, 1, 0, 4,
       1, 0, 4, 1, 0, 4, 1, 4, 1, 0, 4, 1, 0, 4, 1, 0, 4, 1, 0, 4, 1, 0,
       4, 1, 0, 4, 1, 0, 4, 1, 0, 4, 1, 0, 4, 1, 0, 0, 4, 1, 0, 0, 4, 1,
       0, 4, 1, 0, 4, 1, 0, 4, 1, 0, 4, 1, 0, 4, 1, 0, 4, 1, 0, 4, 1, 0,
       4, 1, 0, 4, 1, 0, 4, 1, 0, 4, 1, 0, 4, 1, 0, 4, 1, 0, 4, 1, 0, 4,
       1, 0, 4, 1, 0, 4, 1, 0, 4, 1, 0, 4, 1, 0, 4, 1, 0, 4, 1, 0, 0, 0,
       4, 1, 0, 0, 4, 1, 0, 0, 4, 1, 0, 4, 1, 0, 4, 1, 0, 4, 1, 0, 4, 1,
       0, 4, 1, 0, 4, 1, 0, 4, 1, 0, 4, 1, 0, 4, 1, 0, 4, 1, 0, 4, 1, 0,
       4, 1, 0, 4, 1, 0, 4, 1, 0, 4, 1, 0, 4, 1, 0, 4, 1, 0, 4, 1, 0, 4,
       5], dtype=int32)

In [144]:
smiles = "[CH2]C=C(C)C"
label = Chem.rdinchi.InchiToInchiKey(
    Chem.MolToInchi(Chem.MolFromSmiles(smiles))).strip("-N")
label

'ICPOKTZORRQGLK-UHFFFAOYSA'